# 语言模型

## Ngram

一个由 l 个词构成的句子的概率计算公式为：

$$ p(s) = p(w_1)p(w_2|w_1)p(w_3|w_1 w_2)...p(w_l|w_1..w_{l-1}) \\
        = \prod_{i=1}^l p(w_i | w_1 ...w_{i-1}) \\
        \approx \prod_{i=1}^l p(w_i | wi-1)        $$

上面最后一个例子是二元模型的情况

那么在 bigram 下，给定一个前文 x, y 的概率是多少呢？

$$p(w_n|w_{n-1}) = \frac{C(w_{n-1}w_n)}{\sum_w C(w_{n-1}w)}$$

其实这个式子等于：

$$p(w_n|w_{n-1}) = \frac{C(w_{n-1}w_n)}{C(w_{n-1})}$$

### 代码实现

In [1]:
from collections import OrderedDict, defaultdict, deque, Counter
import json
import random 
import nltk
from nltk import ngrams, FreqDist

In [2]:
words = "It's good to see you you.".split()
words_ngram = list(ngrams(words, 2))
words_freq = FreqDist(words_ngram)

print('words:', words)
print('ngram:', words_ngram)
print('freq:\n')
words_freq

words: ["It's", 'good', 'to', 'see', 'you', 'you.']
ngram: [("It's", 'good'), ('good', 'to'), ('to', 'see'), ('see', 'you'), ('you', 'you.')]
freq:



FreqDist({("It's", 'good'): 1,
          ('good', 'to'): 1,
          ('see', 'you'): 1,
          ('to', 'see'): 1,
          ('you', 'you.'): 1})

### 语言模型

In [3]:
import itertools
import re
import jieba
import more_itertools

In [4]:
def read_data(path):
    with open(path, 'rt') as f:
        for line in f:
            yield line.strip()

def segment_data(data):
    """segment data to sentence."""
    sentence_break = r'(。|！|？)'
    for line in data:
        # 按句子切分，并且捕获分组
        sentences = re.split(sentence_break, line)
        # 捕获的 delimiter 添加到字符串后面
        sentences = itertools.zip_longest(sentences[0::2], 
                                          sentences[1::2],
                                          fillvalue='')
        sentences = (''.join(i) for i in sentences)
        yield sentences

def jieba_cut(sentences):
    # Flat the nested iterator
    sentences = itertools.chain.from_iterable(sentences)
    # Clean empty sentence
    sentences = (sentence for sentence in sentences if sentence)
    for sentence in sentences:
        sentence = deque(jieba.cut(sentence))
        sentence.appendleft('$$')
        sentence = ' '.join(sentence)
        yield sentence

def get_ngram(sentences, n=2):
    """Get ngram from a list of string."""
    for sentence in sentences:
        ngram = ngrams(sentence.split(), n)
        yield ngram
        
def count_ngram(ngram, n=2):
    d = defaultdict(Counter)
    for context, value in ngram:
        d[context][value] += 1
    return d

def count_to_prob(dct):
    prob_dct = dct.copy()
    for context, count in prob_dct.items():
        total = sum(count.values())
        for word in count:
            count[word] /= total
    return prob_dct

def generate_word(context):
    psum = 0
    r = random.random()
    for word, prob in ngram_prob[context].items():
        psum += prob
        if psum > r:
            return word

def generate_sentence(word):
    while word:
        word = generate_word(word)
        yield word

In [5]:
data = read_data('data/YGZ-rain.md')
sentences = segment_data(data)
sentences = jieba_cut(sentences)
ngram = itertools.chain.from_iterable(get_ngram(sentences, n=2))
ngram_counts = count_ngram(ngram)
ngram_prob = count_to_prob(ngram_counts)

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/df/g_gpcyqx3j35s2yvr5k2tysc0000gn/T/jieba.cache
Loading model cost 1.308 seconds.
Prefix dict has been built succesfully.


In [7]:
g_sentence = generate_sentence('$$')
sentence = ''.join(more_itertools.islice_extended(g_sentence, 0, -1))
print(sentence)

不过整个太平洋只为向他听的淡淡土腥气，各成世界小得多可爱，而且躲在一起。


#### 3 个词时候的探索

In [34]:
test = list(ngrams("I don't master regular expression don't master regular expression".split(), 3))
test

[('I', "don't", 'master'),
 ("don't", 'master', 'regular'),
 ('master', 'regular', 'expression'),
 ('regular', 'expression', "don't"),
 ('expression', "don't", 'master'),
 ("don't", 'master', 'regular'),
 ('master', 'regular', 'expression')]

In [33]:
d = defaultdict(Counter)
for *i, j in test:
    d[tuple(i)][j] += 1
    print(i, j)
    
d

['I', "don't"] master
["don't", 'master'] regular
['master', 'regular'] expression
['regular', 'expression'] don't
['expression', "don't"] master
["don't", 'master'] regular
['master', 'regular'] expression


defaultdict(collections.Counter,
            {('I', "don't"): Counter({'master': 1}),
             ("don't", 'master'): Counter({'regular': 2}),
             ('expression', "don't"): Counter({'master': 1}),
             ('master', 'regular'): Counter({'expression': 2}),
             ('regular', 'expression'): Counter({"don't": 1})})

## 平滑

平滑的应用

* 平滑的背景
* 平滑的分类
    + Add one
    + Add k
    + back off / interpolation
    + KN
* 小结

interpolation 相比 back off 效果要好一些

用 Held_out 去设置我们的 lambada，

### Add-one(Laplace) Smoothing

对于unigram模型而言，会有 

$$p_{add1}(w_i) = \frac{C(w_i)+1}{M+|V|}$$

其中，M 是训练语料中所有的N-Gram的数量（token），而 V 是所有的可能的不同的N-Gram的数量（type）

同理，对于bigram模型而言，可得:

$$P_{add1}(w_i|w_{i-1}) = \frac{C(w_{i-1}w_i)+1}{C(w_{i-1})+|V|}$$

推而广之，对于n-Gram模型而言，可得:

$$P_{add1}(w_i|w_{i-n+1},\cdots,w_{i-1})=\frac{C(w_{i-n+1},\cdots,w_i)+1}{C(w_{i-n+1},\cdots,w_{i-1})+|V|}$$

### Add-k Smoothing（Lidstone’s law）

Add k 很简单，就是在 Add-one 的 one 位置 * k，概率公式为：

$$P_{addk}(w_i|w_{i-n+1}\cdots w_{i-1}) = \frac{C(w_{i-n+1}\cdots w_i)+k}{C(w_{i-n+1}\cdots w_{i-1})+k|V|}$$

### Back off

回退的概念是：在高阶模型上找不到是，则回退到低价模型取概率

$$P_{backoff}(w_i|w_{i-n+1}\cdots w_{i-1})=\begin{cases}
P^*(w_i|w_{i-n+1}\cdots w_{i-1})&,if\quad C(w_{i-n+1}\cdots w_i)>0\\
\alpha (w_{i-n+1}\cdots w_{i-1})\cdot P_{backoff}(w_i|w_{i-n+2}\cdots w_{i-1})&,otherwise
\end{cases}$$


### Interpolation

插值算法就是把不同阶别的 n-Gram 模型线性加权组合后再用，公式为：

$$P_{interp}(w_n|w_{n-2},w_{n-1})=\lambda_1P(w_n)+\lambda_2P(w_n|w_{n-1})+\lambda_3P(w_n|w_{n-2},w_{n-1})$$

其中 $0≤λi≤1，∑iλi=1$

### Absolute Discounting

Absolute Discounting 的思想其实跟 Add-one 和 Add-k 差不多, 具体可参考下面的 refrences。

### Kneser-Ney Smoothing

看懂了接续频率会让整个概率变小，但具体的公式暂时没看懂

N-gram 平滑总结：

* Add-1 smoothing:
    + ok for text categorization, not for language modeling
* The most commonly used method:
    + Extended interpolated Kneser-Ney
* For very Large N-grams like the Web:
    * Stupid backoff

## Refrences:



* [https://web.stanford.edu/~jurafsky/slp3/slides/LM_4.pdf](https://web.stanford.edu/~jurafsky/slp3/slides/LM_4.pdf)
* [自然语言处理中N-Gram模型的Smoothing算法 - 白马负金羁 - CSDN博客](http://blog.csdn.net/baimafujinji/article/details/51297802)
* [初涉自然语言处理（五）：平滑处理 | Vkki](http://vkki.me/2016/03/18/nlp-5/)